In [2]:
# necessary
import pandas as pd
import numpy as np
import pickle
from tqdm.auto import tqdm
import os
import ast
import re
import time
import copy
import difflib
from itertools import chain, repeat


from jellyfish import jaro_distance, jaro_winkler
from soynlp.hangle import jamo_levenshtein


import sys
# cur_dir = os.path.dirname(os.path.realpath(__file__))
# root = os.path.abspath(os.path.join(cur_dir, os.pardir, os.pardir))
# src = os.path.abspath(os.path.join(cur_dir, os.pardir))
# temp = root + '/temp'
# sys.path.append(root)
# sys.path.append(src)
# sys.path.append(temp)
    
# from access_database import access_db

def title_comparison(word_0: str, word_1: str) -> tuple:
    
    ''' 
    Compare Product Titles 
    '''
    
    non_sp_0 = word_0.replace(' ', '')
    non_sp_1 = word_1.replace(' ', '')

    len_0 = len(non_sp_0)
    len_1 = len(non_sp_1)
    max_len = max(len_0, len_1)
    min_len = min(len_0, len_1)
    
    # 토큰 중복 비율 계산
    word_sp_0 = word_0.split(' ')
    word_sp_1 = word_1.split(' ')
    
    '''string 종속 여부 체크'''

    leng = min(len(word_sp_0), len(word_sp_1))
    word_set = list(set(word_sp_0) & set(word_sp_1))
    dep_cnt = len(word_set)
    dep_ratio = dep_cnt / leng

    # calculate similarity
    cost = jamo_levenshtein(non_sp_0, non_sp_1)
    sim = (max_len - cost) / max_len

    return round(dep_ratio, 4), dep_cnt, round(sim, 4), max_len, min_len


def prd_mapper(input_data_0: pd.DataFrame, input_data_1: pd.DataFrame) -> pd.DataFrame:
    
    '''  
    Compare Product Titles after Grouping Brands and Categories
    
    Input Data 
    - input_data_0: Mapping Criteria Table
    - input_data_1: Mapping Target Table
    
    ** necessary columns: ['id', 'brand_name', 'product_name', 'category']
    
    Output Data 
    - compared_df: Product name comparison table 
    
    '''
    
    
    # 상품명 NaN값 드랍하기 및 브랜드명 전처리 
    df_notnull_0 = input_data_0[input_data_0.product_name.notnull()].reset_index(drop=True)
    df_notnull_0.loc[:, 'brand_name'] = df_notnull_0.brand_name.str.replace(' ', '').str.lower()
    df_notnull_1 = input_data_1[input_data_1.product_name.notnull()].reset_index(drop=True)
    df_notnull_1.loc[:, 'brand_name'] = df_notnull_1.brand_name.str.replace(' ', '').str.lower()
    

    columns = ['id_1', 'id_0', 'title_1', 'title_0', 'brand_name', 'category', 'table_name', 'dependency_ratio', 'dependency_count', 'similarity', 'max_length', 'min_length']
    compared_df = pd.DataFrame(columns=columns)

    # group by brand_name
    brd_grp_0 = df_notnull_0.groupby('brand_name')
    brd_grp_1 = df_notnull_1.groupby('brand_name')
    brands = brd_grp_1.size().index.tolist()
    for brand in tqdm(brands):
        df_brd_1 = brd_grp_1.get_group(brand).reset_index(drop=True)
        
        try:
            df_brd_0 = brd_grp_0.get_group(brand).reset_index(drop=True)
    
    
        except KeyError: # 매핑 기준 테이블에 해당 브랜드가 존재하지 않는 경우 
            continue
        
        
        # group by category
        categ_grp_0 = df_brd_0.groupby('category')
        categ_grp_1 = df_brd_1.groupby('category')
        categs = categ_grp_1.size().index.tolist()
        for categ in categs:
            df_categ_1 = categ_grp_1.get_group(categ).reset_index(drop=True)
            
            try:
                df_categ_0 = categ_grp_0.get_group(categ).reset_index(drop=True)


            except KeyError: # 매핑 기준 테이블에 해당 카테고리가 존재하지 않는 경우 
                continue
            
            
            for idx_1 in range(len(df_categ_1)):
                id_1, title_1, tbl = df_categ_1.loc[idx_1, ['id', 'product_name', 'table_name']]
                
                
                for idx_0 in range(len(df_categ_0)):
                    id_0, title_0 = df_categ_0.loc[idx_0, ['id', 'product_name']]
                    
                    compare_output = title_comparison(title_1, title_0)
                    compared_df.loc[len(compared_df)] = (id_1, id_0, title_1, title_0, brand, categ, tbl) + compare_output
                
    
    compared_df = compared_df.reset_index(drop=True)
                
    return compared_df


def select_mapped_prd(input_data: pd.DataFrame) -> pd.DataFrame:
    
    '''  
    Select Mapped Products by Criteria You Define
    
    Input Data 
    - input_data: Product name comparison table
    
    ** necessary columns = ['id_1', 'id_0', 'title_1', 'title_0', 'brand_name', 'category', 'table_name', 'dependency_ratio', 'dependency_count', 'similarity', 'max_length', 'min_length']
    
    Output Data 
    - mapped_df: Product Name Mapping Complete Table
    
    '''
    
    params = {
        'min_length': 6, # product name minimum length
        'min_token': 3, # product name token minimum length
        'levenshtein_similarity': round(5/6, 4) # similarity minimum value
    }

    min_len = params['min_length']
    min_tk = params['min_token']
    min_sim = params['levenshtein_similarity']
    
    compared_df = input_data.copy()
    # mapping group 0: simimlarity == 1 & min_length >= min_len
    grp_0 = compared_df[(compared_df.similarity==1) & (compared_df.min_length>=min_len)]
    grp_sim = grp_0.sort_values(by=['id_1', 'dependency_ratio', 'dependency_count', 'similarity'], ascending=False).reset_index(drop=True)

    # mapping group 1: simimlarity != 1 & dependency_ratio = 1 & min_length >= min_len
    grp_1 = compared_df[(compared_df.similarity!=1) & (compared_df.dependency_ratio==1) & (compared_df.min_length>=min_len)].sort_values(by=['id_1', 'dependency_count', 'similarity'], ascending=False)
    grp_dedup = grp_1.drop_duplicates(subset=['id_1'], keep='first').reset_index(drop=True)

    # mapping group 2: simimlarity != 1 & dependency_ratio != 1 & min_length >= min_len
    df_dedup = pd.concat([compared_df, grp_sim, grp_dedup]).drop_duplicates(subset=['id_1'], keep=False).reset_index(drop=True)
    grp_2 = df_dedup[df_dedup.min_length>=min_len].reset_index(drop=True)

    # calculate levenshtein distance
    for idx in tqdm(range(len(grp_2))):
        s0 = grp_2.loc[idx, 'title_0'].replace(' ', '')
        s1 = grp_2.loc[idx, 'title_1'].replace(' ', '')
        winkler = jaro_winkler(s0, s1)
        grp_2.loc[idx, 'jaro_winker'] = round(winkler, 4)
    
    # mapping group 3: simimlarity != 1 & dependency_ratio != 1 & min_length >= min_len & levenshtein_similarity >= min_sim
    grp_levenshtein = grp_2.loc[(grp_2.similarity>=min_sim)].reset_index(drop=True)

    mapped_df = pd.concat([grp_sim, grp_dedup, grp_levenshtein]).sort_values('id_1').reset_index(drop=True)

    return mapped_df


def md_map_tbl(input_data: pd.DataFrame) -> pd.DataFrame:
    
    # with open('./config/info_table_id_dict.txt', 'rb') as f:
    #     info_table_id_dict = pickle.load(f)    
    # table_id = info_table_id_dict[table_name]
    
    mapping_table = pd.DataFrame(columns=['glowpick_product_info_final_version_id', 'mapped_id', 'table_name'])
    for table_name in input_data.table_name.unique():
        
        mapped_df = input_data.loc[input_data.table_name==table_name].reset_index(drop=True)

        for id_0 in tqdm(mapped_df.id_0.unique()):

            ids = mapped_df.loc[mapped_df.id_0==id_0, 'id_1'].values.tolist()

            if len(ids) == 1:
                ids = str(ids[0])

            else:
                ids = str(ids)

            mapping_table.loc[len(mapping_table)] = int(id_0), ids, table_name

    
    return mapping_table


def concat_map_tbl():
    
    ''' new mapping table concat '''

    mapping_table = pd.DataFrame()
    
    files = [f for f in os.listdir(temp) if re.match(r'mapping_table_[0-9].csv', f)]
    for table in files:
        tbl = pd.read_csv(temp + f'/{table}')
        mapping_table = pd.concat([mapping_table, tbl])
    mapping_table = mapping_table.reset_index(drop=True)
    
    mapping_table.to_csv(temp + '/mapping_table.csv', index=False)
    
    return None


def update_map_tbl(user_name, password, db_name):
    
    ''' update mapping table '''
    db = access_db.AccessDataBase(user_name, password, db_name)
    
    # get existing mapping table to db
    map_tbl_ex = db.get_tbl(db_name, 'naver_glowpick_mapping_table', 'all') 
    # get new mapping table to dir(temp)
    map_tbl_new = pd.read_csv(temp + '/mapping_table.csv')
    
        
    df_concat = pd.concat([map_tbl_ex, map_tbl_new]).reset_index(drop=True)

    # updated mapping table
    map_tbl = pd.DataFrame(columns=map_tbl_new.columns)

    for tbl in df_concat.table_name.unique():
        df_tbl = df_concat[df_concat.table_name==tbl]
        
        for id_ in tqdm(df_tbl.glowpick_product_info_final_version_id.unique()):
            mapped_id = df_tbl.loc[df_tbl.glowpick_product_info_final_version_id==id_, 'mapped_id']
            
            mapped_ids = []
            for ids in mapped_id:
                if ids[0] == '[':
                    ids = ast.literal_eval(ids)
                    mapped_ids += ids
                
                else:
                    mapped_ids.append(int(ids))
            
            mapped_ids = list(set(mapped_ids))
            if len(mapped_ids) == 0:
                print('wrn')
                break
            
            elif len(mapped_ids) == 1:
                mapped_ids = str(mapped_ids[0])

            else:
                mapped_ids = str(mapped_ids)

                
            map_tbl.loc[len(map_tbl)] = int(id_), str(mapped_ids), str(tbl)
            
    map_tbl = map_tbl.sort_values(by=['glowpick_product_info_final_version_id', 'table_name']).reset_index(drop=True)
    map_tbl.to_csv(temp + '/naver_glowpick_mapping_table.csv', index=False)

    # dup check
    dup = map_tbl[map_tbl.duplicated(subset=['glowpick_product_info_final_version_id', 'table_name'], keep=False)]
    if len(dup) == 0:
        comment = 'Mapping table update complete!'
    
    else:
        comment = 'Duplicate check required!'
    
    unq_prd_0 = len(map_tbl.glowpick_product_info_final_version_id.unique())
    unq_prd_1 = (map_tbl.mapped_id.str.count(',') + 1).sum()
    
    return comment, unq_prd_0, unq_prd_1
    
    
def upload_map_tbl(user_name, password, db_name, table_name, columns):
    
    ''' upload mapping table 
    
    <업로드 조건> 
    ** 기존 매핑 테이블 오늘 날짜로 백업 완료
    ** db에서 매핑테이블 생성 및 컬럼명 일치 여부 확인 
     
    '''
    db = access_db.AccessDataBase(user_name, password, db_name)
    
    map_tbl = pd.read_csv(temp + '/naver_glowpick_mapping_table.csv').loc[:, columns]
    
    db.engine_upload(map_tbl, db_name, table_name)
    
    comment = 'Mapping table upload is complete!'
    
    return comment

In [95]:
# df_0 = pd.read_csv('../tbl_cache/deprepro_0.csv')
# df_1 = pd.read_csv('../tbl_cache/deprepro_1.csv')

def prd_mapper_():
    
    df_0 = pd.read_csv('../tbl_cache/deprepro_0.csv')
    df_1 = pd.read_csv('../tbl_cache/deprepro_1.csv')

    # concat, drop category null, length condition check
    df_concat = pd.concat([df_0, df_1])
    df_concat_ = df_concat[df_concat.category.notnull()].reset_index(drop=True)
    df_concat_.loc[:, 'product_name_'] = df_concat_.product_name.str.replace(' ', '')
    df_concat_.loc[:, 'length'] = df_concat_.product_name_.str.len()
    df_len = df_concat_[df_concat_.length >= 6].reset_index(drop=True)

    # find duplicate
    subset = ['brand_name', 'category', 'product_name_']
    df_dup = df_len[df_len.duplicated(subset=subset, keep=False)]

    # grouping 
    df_grp = df_dup.groupby(subset)
    grp_index = df_grp.size().index


    # 브랜드, 카테고리, 상품명 모두 일치하는 상품들 그룹핑 후 매핑 
    map_list, map_list_ = [], []
    for idx in tqdm(grp_index):
        # 하나의 (brand_name, category, product_name) 유니크 그룹
        df = df_grp.get_group(idx).reset_index(drop=True)

        # 글로우픽 기준으로 매핑 작업 
        if 'glowpick_product_info_final_version' in df.table_name.tolist():
            id_ = df.loc[df.table_name=='glowpick_product_info_final_version'].id.tolist()[0]  
            title = df.loc[0, 'product_name']
            brand = df.loc[0, 'brand_name']
            categ = df.loc[0, 'category']
            length = len(title.replace(' ', ''))

            tbls = df.loc[df.table_name!='glowpick_product_info_final_version'].table_name.unique().tolist()
            for tbl in tbls:
                mapped_ids = df.loc[df.table_name==tbl].id.tolist()

                if len(mapped_ids) == 0:
                    continue

                elif len(mapped_ids) == 1:
                    mapped_id = int(mapped_ids[0])
                    map_list.append([int(mapped_id), int(id_), str(title), str(title),  str(brand), str(categ), str(tbl), np.nan, np.nan, 1, length, length])

                    mapped_id = str(mapped_ids[0])
                else:
                    for mapped_id in mapped_ids:
                        map_list.append([int(mapped_id), int(id_), str(title), str(title),  str(brand), str(categ), str(tbl), np.nan, np.nan, 1, length, length])

                    mapped_id = str(mapped_ids)

                map_list_.append([int(id_), str(mapped_id), str(tbl)])

        else:
            pass

    columns=['glowpick_product_info_final_version_id', 'mapped_id', 'table_name'] 
    df_map = pd.DataFrame(map_list_, columns=columns)

    # 전체 데이터에서 매핑된 데이터 제거 (차집합 구하기)
    df_map_ = map_expand(df_map)
    df_concat = pd.concat([df_1, df_map_.loc[:, dup_col]])
    df_dedup = df_concat.drop_duplicates(subset=dup_col, keep=False).reset_index(drop=True)

    return map_list, df_dedup


# # sorting
# df_map = df_map.sort_values(by=['glowpick_product_info_final_version_id', 'table_name']).reset_index(drop=True)

In [96]:
prds = prd_mapper_()

  0%|          | 0/2427 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [98]:
prds[1]

,id,brand_name,product_name,selection,division,groups,table_name,category
0,21761,1.618,워터리즘 히알루론산 토너,화장품/미용,스킨케어,스킨/토너,naver_beauty_product_info_extended_v1_211217,스킨/토너
1,24796,100%퓨어,프루트 피그먼티드 립 글로즈,화장품/미용,색조메이크업,립글로스,naver_beauty_product_info_extended_v1_211217,립메이크업
2,21765,16BRAND,브랜드 마이 매거진,화장품/미용,색조메이크업,아이섀도,naver_beauty_product_info_extended_v1_211217,아이섀도
3,21766,16BRAND,브랜드 캔디락 슈가 파티,화장품/미용,색조메이크업,아이섀도,naver_beauty_product_info_extended_v1_211217,아이섀도
4,21767,16BRAND,브랜드 브릭킷 섀도우 골든 바 에디션,화장품/미용,색조메이크업,아이섀도,naver_beauty_product_info_extended_v1_211217,아이섀도
...,...,...,...,...,...,...,...,...
16692,23006,힌스,뉴 뎁스 아이섀도우 얼루어 인 모션,화장품/미용,색조메이크업,아이섀도,naver_beauty_product_info_extended_v1_211217,아이섀도
16693,23012,힌스,세컨 스킨 파운데이션 spf30,화장품/미용,베이스메이크업,파운데이션,naver_beauty_product_info_extended_v1_211217,베이스메이크업
16694,21308,힐시드,슈퍼스타 아쿠아 크림,화장품/미용,스킨케어,크림,naver_beauty_product_info_extended_v1_211217,크림
16695,21309,힐시드,아쿠아 밸런싱 시럽 토너,화장품/미용,스킨케어,스킨/토너,naver_beauty_product_info_extended_v1_211217,스킨/토너


In [83]:
with open('../tbl_cache/mapped_list.txt', 'wb') as f:
    pickle.dump(map_list, f)

In [84]:
len0 = len(df_map[df_map.duplicated(subset=['glowpick_product_info_final_version_id', 'table_name'], keep=False)])
len1 = len(df_map[df_map.duplicated(subset=['mapped_id', 'table_name'], keep=False)])
if len0 + len1 == 0:
    print('\n\tComplete!!')


	Complete!!


In [85]:
def map_expand(mapping_table: pd.DataFrame) -> pd.DataFrame:
    mapping_table = mapping_table.reset_index(drop=True)
    
    map_list = []
    for idx in tqdm(range(len(mapping_table))):
        id_0 = mapping_table.iloc[idx, 0]
        ids = mapping_table.iloc[idx, 1]
        tbl = mapping_table.iloc[idx, 2]
        
        if ids[0] == '[':
            ids = ast.literal_eval(ids)
            for id_1 in ids:
                map_list.append([int(id_0), int(id_1), str(tbl)])
                # mapping_.loc[len(mapping_)] = int(id_0), int(id_1), str(tbl)

        else:
            id_1 = ids
            map_list.append([int(id_0), int(id_1), str(tbl)])
            # mapping_.loc[len(mapping_)] = int(id_0), int(id_1), str(tbl)
    
    mapping_ = pd.DataFrame(map_list, columns=['item_key', 'id', 'table_name'])
    return mapping_

In [86]:
df_map_ = map_expand(df_map)
df_concat = pd.concat([df_1, df_map_.loc[:, dup_col]])
df_dedup = df_concat.drop_duplicates(subset=dup_col, keep=False).reset_index(drop=True)

  0%|          | 0/8224 [00:00<?, ?it/s]

In [93]:
df_map_

,item_key,id,table_name
0,33334,21762,naver_beauty_product_info_extended_v1_211217
1,69374,21800,naver_beauty_product_info_extended_v1_211217
2,69407,21792,naver_beauty_product_info_extended_v1_211217
3,69359,21797,naver_beauty_product_info_extended_v1_211217
4,69363,21789,naver_beauty_product_info_extended_v1_211217
...,...,...,...
9442,23104,23014,naver_beauty_product_info_extended_v1_211217
9443,23092,23013,naver_beauty_product_info_extended_v1_211217
9444,25586,23010,naver_beauty_product_info_extended_v1_211217
9445,23101,2642,naver_beauty_product_info_extended_v1_211217


In [87]:
subset = ['id', 'table_name']
df_map_[df_map_.duplicated(subset=dup_col, keep=False)]

,item_key,id,table_name


In [88]:
df_concat = pd.concat([df_1, df_map_.loc[:, dup_col]])
df_dedup = df_concat.drop_duplicates(subset=dup_col, keep=False).reset_index(drop=True)

In [89]:
if len(df_1) - len(df_map_) == len(df_dedup):
    print('complete!!')

complete!!


In [90]:
df_dedup[df_dedup.duplicated(subset=dup_col, keep=False)]

,id,brand_name,product_name,selection,division,groups,table_name,category


In [91]:
df_dedup

,id,brand_name,product_name,selection,division,groups,table_name,category
0,21761,1.618,워터리즘 히알루론산 토너,화장품/미용,스킨케어,스킨/토너,naver_beauty_product_info_extended_v1_211217,스킨/토너
1,24796,100%퓨어,프루트 피그먼티드 립 글로즈,화장품/미용,색조메이크업,립글로스,naver_beauty_product_info_extended_v1_211217,립메이크업
2,21765,16BRAND,브랜드 마이 매거진,화장품/미용,색조메이크업,아이섀도,naver_beauty_product_info_extended_v1_211217,아이섀도
3,21766,16BRAND,브랜드 캔디락 슈가 파티,화장품/미용,색조메이크업,아이섀도,naver_beauty_product_info_extended_v1_211217,아이섀도
4,21767,16BRAND,브랜드 브릭킷 섀도우 골든 바 에디션,화장품/미용,색조메이크업,아이섀도,naver_beauty_product_info_extended_v1_211217,아이섀도
...,...,...,...,...,...,...,...,...
16692,23006,힌스,뉴 뎁스 아이섀도우 얼루어 인 모션,화장품/미용,색조메이크업,아이섀도,naver_beauty_product_info_extended_v1_211217,아이섀도
16693,23012,힌스,세컨 스킨 파운데이션 spf30,화장품/미용,베이스메이크업,파운데이션,naver_beauty_product_info_extended_v1_211217,베이스메이크업
16694,21308,힐시드,슈퍼스타 아쿠아 크림,화장품/미용,스킨케어,크림,naver_beauty_product_info_extended_v1_211217,크림
16695,21309,힐시드,아쿠아 밸런싱 시럽 토너,화장품/미용,스킨케어,스킨/토너,naver_beauty_product_info_extended_v1_211217,스킨/토너


In [92]:
df_dedup.to_csv('../tbl_cache/deprepro_1.csv', index=False)

In [44]:
df_map.to_csv('../tbl_cache/df_map.csv', index=False)

In [45]:
df_map

,glowpick_product_info_final_version_id,mapped_id,table_name
0,34,11,naver_beauty_product_info_extended_v1_211217
1,35,15,naver_beauty_product_info_extended_v1_211217
2,88,"[46, 52]",naver_beauty_product_info_extended_v1_211217
3,89,29,naver_beauty_product_info_extended_v1_211217
4,93,54,naver_beauty_product_info_extended_v1_211217
...,...,...,...
8219,111437,9332,naver_beauty_product_info_extended_v1_211217
8220,111456,"[22970, 22985]",naver_beauty_product_info_extended_v1_211217
8221,111460,14441,naver_beauty_product_info_extended_v1_211217
8222,111478,12393,naver_beauty_product_info_extended_v1_211217
